In [1]:
import os

os.chdir("C:\\Users\\CeNTREX\\Documents\\GitHub\\1-3-IR-high_voltage\\Short IR electrodes test")

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
from RadEyeG20 import * 
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares
import winsound
import nidaqmx
import pandas as pd

# If the imports don't work, press F1, find "Developer: Reload Window"

factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title4

#rm = pyvisa.ResourceManager()
#rm.list_resources()


In [2]:
import os

os.chdir("C:\\Users\\CeNTREX\\Documents\\GitHub\\1-3-IR-high_voltage\\Short IR electrodes test\\4-1-2025_test")

In [2]:
device_name = "dev1"
DAQ_read_channels = ["ai0","ai1","ai2", "ai3"]
DAQ_write_channel = "ao0"
DAQ_timeout = 5000

### Data collection functions

In [3]:
# RAMP TO VOLTAGE 


def ramp_to(start_v, end_v, total_ramp_time=60, v_step=0.001):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.
    
    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return
    
    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(voltage_range / v_step)  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step
    print(dt)
    
    # Set the initial voltage to start_v
    current_voltage = start_v

    try:
        with nidaqmx.Task() as task:
            # Configure the AO0 channel on device "dev1"
            task.ao_channels.add_ao_voltage_chan("dev1/ao0", min_val=0.0, max_val=10.1)
            
            # Set the initial voltage to start_v
            task.write(current_voltage)
            print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")
            
            # Determine the direction of the ramp (up or down)
            step_direction = 1 if end_v > start_v else -1
            
            for _ in range(num_steps):
                # Update the current voltage in the direction of the ramp
                current_voltage += step_direction * v_step
                
                # Output the new voltage to the DAQ
                task.write(current_voltage)
                print(f"current voltage: {current_voltage}V.")
                
                # Wait for the calculated time step
                time.sleep(dt)
                
                # Placeholder for additional error checking or exception handling
                # You can add custom error handling code here
                pass

    except Exception as e:
        # If any exception occurs, handle it here
        print(f"An exception occurred: {str(e)}. Exiting ramp.")
        return

    print(f"Successfully ramped from {start_v}V to {current_voltage}V.")


# Example usage:
# ramp_to(0.5, 5.0)  # Ramps from 0.5V to 5.0V over 60 seconds with a step size of 0.001V




# RAMP TO BUT SAVE DAQ AND MULTIMETER DATA

def ramp_and_record(start_v, end_v, measure_time, total_ramp_time=60, v_step=0.001, filename="ramp_and_record"):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.

    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return

    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(np.ceil(voltage_range / v_step))  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step

    # Set the initial voltage to start_v
    current_voltage = start_v

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)*((num_steps + 1)/(num_steps))))

    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Start the DAQ task for data logging
        read_task.start()
        


        try:
            with nidaqmx.Task() as task:
                # Configure the AO0 channel on device "dev1"
                task.ao_channels.add_ao_voltage_chan("dev1/ao0", min_val=0.0, max_val=10.1)

                # Set the initial voltage to start_v
                task.write(current_voltage)
                print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")

                # Determine the direction of the ramp (up or down)
                step_direction = 1 if end_v > start_v else -1

                for _ in range(num_steps):

                    temp_timer = time.time()

                    #Wait for the calculated time step
                    time.sleep(dt)

                    # Update the current voltage in the direction of the ramp
                    current_voltage += step_direction * v_step

                    # Output the new voltage to the DAQ
                    task.write(current_voltage)
                    print(f"current voltage: {current_voltage}V.")

                    # Placeholder for additional error checking or exception handling
                    pass

        except Exception as e:
            # If any exception occurs, handle it here
            print(f"An exception occurred: {str(e)}. Exiting ramp.")
            return
        
        temp_timer = time.time()

        print(f"Successfully ramped from {start_v}V to {current_voltage}V.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon_neg': DAQ_data[0],
        'I_mon_neg': DAQ_data[1],
        'V_mon_pos': DAQ_data[2],
        'I_mon_pos': DAQ_data[3]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{measure_time}.csv"
    df.to_csv(full_file_name, index=False)

# Example usage:
# ramp_and_record(0.1, 0, 30, 0.001)  # Ramps from 0.1V to 0V over 30 seconds with a step size of 0.001V


def record(total_ramp_time=60, measure_time = 0, filename="record"):

    # DMM settings
    t0=time.time()

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)))
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')
  

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Start the DAQ task for data logging
        read_task.start()
        

        print(f"Successfully recorded.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon_neg': DAQ_data[0],
        'I_mon_neg': DAQ_data[1],
        'V_mon_pos': DAQ_data[2],
        'I_mon_pos': DAQ_data[3]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_recording_{total_ramp_time}s_{measure_time}.csv"   
    df.to_csv(full_file_name, index=False)

def ramp_multi(min_v, max_v, tot_measure_time, total_rampup_time=60,total_rampdown_time=60, v_step=0.01):
    
    for i in range(tot_measure_time):

        ramp_and_record(min_v,max_v, i, total_rampup_time, v_step)

        record(300,i,"rec_at_10_")

        ramp_and_record(max_v,min_v, i, total_rampdown_time, v_step)

        record(60,i,"rec_at_0_")

    


In [ ]:
os.chdir("C:\\Users\\CeNTREX\\Documents\\GitHub\\1-3-IR-high_voltage\\Short IR electrodes test\\4-3-2025_test")

ramp_multi(min_v=0., max_v= 10., tot_measure_time=10, total_rampup_time=120, total_rampdown_time=60, v_step=0.01)

Starting ramp from 0.0V to 10.0V with step 0.01V.
current voltage: 0.01V.
current voltage: 0.02V.
current voltage: 0.03V.
current voltage: 0.04V.
current voltage: 0.05V.
current voltage: 0.060000000000000005V.
current voltage: 0.07V.
current voltage: 0.08V.
current voltage: 0.09V.
current voltage: 0.09999999999999999V.
current voltage: 0.10999999999999999V.
current voltage: 0.11999999999999998V.
current voltage: 0.12999999999999998V.
current voltage: 0.13999999999999999V.
current voltage: 0.15V.
current voltage: 0.16V.
current voltage: 0.17V.
current voltage: 0.18000000000000002V.
current voltage: 0.19000000000000003V.
current voltage: 0.20000000000000004V.
current voltage: 0.21000000000000005V.
current voltage: 0.22000000000000006V.
current voltage: 0.23000000000000007V.
current voltage: 0.24000000000000007V.
current voltage: 0.25000000000000006V.
current voltage: 0.26000000000000006V.
current voltage: 0.2700000000000001V.
current voltage: 0.2800000000000001V.
current voltage: 0.29000

In [41]:
os.chdir("C:\\Users\\CeNTREX\\Documents\\GitHub\\1-3-IR-high_voltage\\Short IR electrodes test\\4-2-2025_test")
ramp_and_record(0.,10.,measure_time = 1,total_ramp_time=1800, v_step=0.001)

Starting ramp from 0.0V to 10.0V with step 0.001V.
current voltage: 0.001V.
current voltage: 0.002V.
current voltage: 0.003V.
current voltage: 0.004V.
current voltage: 0.005V.
current voltage: 0.006V.
current voltage: 0.007V.
current voltage: 0.008V.
current voltage: 0.009000000000000001V.
current voltage: 0.010000000000000002V.
current voltage: 0.011000000000000003V.
current voltage: 0.012000000000000004V.
current voltage: 0.013000000000000005V.
current voltage: 0.014000000000000005V.
current voltage: 0.015000000000000006V.
current voltage: 0.016000000000000007V.
current voltage: 0.017000000000000008V.
current voltage: 0.01800000000000001V.
current voltage: 0.01900000000000001V.
current voltage: 0.02000000000000001V.
current voltage: 0.02100000000000001V.
current voltage: 0.022000000000000013V.
current voltage: 0.023000000000000013V.
current voltage: 0.024000000000000014V.
current voltage: 0.025000000000000015V.
current voltage: 0.026000000000000016V.
current voltage: 0.02700000000000

In [42]:
record(total_ramp_time=3600, measure_time = 0, filename="rec_at_10_")

ramp_and_record(10.,0.,measure_time = 1,total_ramp_time=1800, v_step=0.001)

Successfully recorded.
Starting ramp from 10.0V to 0.0V with step 0.001V.
current voltage: 9.999V.
current voltage: 9.998000000000001V.
current voltage: 9.997000000000002V.
current voltage: 9.996000000000002V.
current voltage: 9.995000000000003V.
current voltage: 9.994000000000003V.
current voltage: 9.993000000000004V.
current voltage: 9.992000000000004V.
current voltage: 9.991000000000005V.
current voltage: 9.990000000000006V.
current voltage: 9.989000000000006V.
current voltage: 9.988000000000007V.
current voltage: 9.987000000000007V.
current voltage: 9.986000000000008V.
current voltage: 9.985000000000008V.
current voltage: 9.984000000000009V.
current voltage: 9.98300000000001V.
current voltage: 9.98200000000001V.
current voltage: 9.98100000000001V.
current voltage: 9.980000000000011V.
current voltage: 9.979000000000012V.
current voltage: 9.978000000000012V.
current voltage: 9.977000000000013V.
current voltage: 9.976000000000013V.
current voltage: 9.975000000000014V.
current voltage:

In [22]:
ramp_to(9.,10.,total_ramp_time=120,v_step=0.005)

0.6
Starting ramp from 9.0V to 10.0V with step 0.005V.
current voltage: 9.005V.
current voltage: 9.010000000000002V.
current voltage: 9.015000000000002V.
current voltage: 9.020000000000003V.
current voltage: 9.025000000000004V.
current voltage: 9.030000000000005V.
current voltage: 9.035000000000005V.
current voltage: 9.040000000000006V.
current voltage: 9.045000000000007V.
current voltage: 9.050000000000008V.
current voltage: 9.055000000000009V.
current voltage: 9.06000000000001V.
current voltage: 9.06500000000001V.
current voltage: 9.070000000000011V.
current voltage: 9.075000000000012V.
current voltage: 9.080000000000013V.
current voltage: 9.085000000000013V.
current voltage: 9.090000000000014V.
current voltage: 9.095000000000015V.
current voltage: 9.100000000000016V.
current voltage: 9.105000000000016V.
current voltage: 9.110000000000017V.
current voltage: 9.115000000000018V.
current voltage: 9.120000000000019V.
current voltage: 9.12500000000002V.
current voltage: 9.13000000000002V.

In [15]:
ramp_and_record(0,0.5,0,300,0.05)

Starting ramp from 0V to 0.5V with step 0.05V.


KeyboardInterrupt: 

In [2]:
import pyvisa

# Create a resource manager
rm = pyvisa.ResourceManager()

# List all available VISA resources
resources = rm.list_resources()

# Print only COM ports (ASRL resources)
com_ports = [r for r in resources if "ASRL" in r or "COM" in r]
print("Available COM ports:", com_ports)

Available COM ports: ['ASRL3::INSTR', 'ASRL4::INSTR']


In [3]:
import time

In [4]:
import pyvisa 
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL4::INSTR')

In [5]:
# Example usage
time_offset = 1
com_port =  'ASRL4::INSTR'  # Replace with detected COM port
radeye = RadEyeG20(time_offset , com_port)
print(radeye.ReadValue())

[1743723532.2861886, 80]


In [8]:

radeye.__exit__()

In [6]:
for i in range (10):
    time_offset = time.time()
    time.sleep(0.1)
    print(radeye.ReadValue())


[1743723534.1822236, 80]
[1743723534.4005835, 70]
[1743723535.4073017, 80]
[1743723536.3982413, 80]
[1743723537.405982, 80]
[1743723538.3963506, 80]
[1743723539.4030638, 80]
[1743723540.3945024, 80]
[1743723541.4011621, 80]
[1743723542.3919199, 80]


In [7]:
1

1